In [51]:
import pandas as pd
import numpy as np

In [52]:
fl = pd.read_csv('checkins.dat', sep='|', skipinitialspace=True)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
fl = fl.drop(['id    ', 'user_id ', 'venue_id ', 'created_at      '], axis=1)
fl = fl.dropna()
fl = fl.replace(r'^\s+$', np.nan, regex=True)
fl = fl.rename(index=str, columns={'latitude      ': 'latitude', 'longitude     ': 'longitude'})
fl.columns

Index(['latitude', 'longitude'], dtype='object')

In [54]:
from sklearn.cluster import MeanShift

In [55]:
ms = MeanShift(bandwidth=0.1)

In [56]:
data = fl.values
data = data[0:100000]
print(data)

[[  38.8951118  -77.0363658]
 [  33.800745   -84.41052  ]
 [  45.5234515 -122.6762071]
 ...
 [  29.7628844  -95.3830615]
 [  32.802955   -96.769923 ]
 [  37.7749295 -122.4194155]]


In [79]:
ms.fit(data)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [85]:
cluster_centers = ms.cluster_centers_
print(cluster_centers)
print('length:', len(cluster_centers))

[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ...
 [  38.891565   -121.2930079 ]
 [  42.5953378   -78.9411461 ]
 [  41.5822716   -85.8344383 ]]
length: 3230


In [87]:
labels = ms.labels_
print(labels)
print('length:', len(labels))

[ 5  7 30 ... 25 19  4]
length: 100000


In [109]:
match = []
for i in range(len(cluster_centers)):
    count = 0
    for l in labels:
        if i == l:
            count += 1
    if count > 15:
        match.append(i)
    else:
        match.append(-1)

In [98]:
offices = [[33.751277, -118.188740], [25.867736, -80.324116],
          [51.503016, -0.075479], [52.378894, 4.885084],
          [39.366487, 117.036146], [-33.868457, 151.205134]]

In [106]:
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
print(n_clusters_)

3230


In [122]:
min_dist = -1
min_dist_list = {}
for m in match:
    if m == -1:
        continue
    
    x1 = cluster_centers[m][0]
    x2 = cluster_centers[m][1]
    
    min_dist = -1
    for o in offices:
        o1 = o[0]
        o2 = o[1]
        
        dist = np.sqrt((x1 - o1)**2 + (x2 - o2)**2)
        
        if min_dist == -1 or min_dist > dist:
            min_dist = dist
            
    min_dist_list[min_dist] = cluster_centers[m]

In [123]:
result = min_dist_list[min(min_dist_list)]

In [127]:
def write(result):
    with open('submit.txt', 'w') as file:
        file.write('{} {}'.format(result[0], result[1]))

In [128]:
write(result)